In [2]:
import random

# Define the parameters
N = 8  # Size of the chessboard
population_size = 100  # Number of individuals in the population
mutation_rate = 0.1  # Probability of mutation

# Define the chessboard weights (randomly generated for this example)
weights = [[random.randint(1, 10) for _ in range(N)] for _ in range(N)]


def calculate_fitness(permutation):
    # Calculate the fitness of a permutation
    total_weight = 0
    for row, col in enumerate(permutation):
        total_weight += weights[row][col]
        for i in range(row + 1, N):
            if abs(row - i) == abs(col - permutation[i]):
                # Penalize if queens threaten each other
                total_weight -= 10
    return total_weight


def generate_initial_population():
    # Generate an initial population of permutations randomly
    population = []
    for _ in range(population_size):
        permutation = random.sample(range(N), N)
        population.append(permutation)
    return population


def selection(population):
    # Tournament selection
    selected = []
    for _ in range(population_size):
        tournament = random.sample(population, 2)
        fitness_values = [calculate_fitness(permutation) for permutation in tournament]
        selected.append(tournament[fitness_values.index(max(fitness_values))])
    return selected


def crossover(parent1, parent2):
    # Single-point crossover
    crossover_point = random.randint(1, N - 1)
    child = parent1[:crossover_point] + parent2[crossover_point:]
    return child


def mutation(individual):
    # Swap mutation
    if random.random() < mutation_rate:
        pos1, pos2 = random.sample(range(N), 2)
        individual[pos1], individual[pos2] = individual[pos2], individual[pos1]

    # Fix duplicate values
    duplicates = set([x for x in individual if individual.count(x) > 1])
    while duplicates:
        for i in range(N):
            if individual[i] in duplicates:
                available_values = set(range(N)).difference(individual)
                if available_values:
                    individual[i] = random.choice(list(available_values))
                else:
                    individual[i] = random.randint(0, N-1)
        duplicates = set([x for x in individual if individual.count(x) > 1])

    return individual


def genetic_algorithm():
    # Generate the initial population
    population = generate_initial_population()

    # Evolution loop
    for _ in range(100):
        # Select individuals for reproduction
        selected_population = selection(population)

        # Create new offspring population through crossover and mutation
        offspring_population = []
        while len(offspring_population) < population_size:
            parent1, parent2 = random.sample(selected_population, 2)
            child = crossover(parent1, parent2)
            child = mutation(child)
            offspring_population.append(child)

        # Calculate fitness for the new population
        fitness_values = [calculate_fitness(permutation) for permutation in offspring_population]

        # Form the new population based on fitness
        population = [permutation for _, permutation in sorted(zip(fitness_values, offspring_population), reverse=True)]

    # Return the best-performing individual (permutation)
    return population[0]


# Function to print the chessboard with queens' positions
def print_chessboard(positions):
    for row in range(N):
        line = ""
        for col in range(N):
            if positions[row] == col:
                line += "Q "
            else:
                line += "- "
        print(line)
    print()


# Run the genetic algorithm and print the solution
solution = genetic_algorithm()
print("Best solution:")
print_chessboard(solution)
print("Fitness:", calculate_fitness(solution))

Best solution:
- - - - Q - - - 
- - - - - - - Q 
- - - - - Q - - 
- - Q - - - - - 
- - - - - - Q - 
- Q - - - - - - 
- - - Q - - - - 
Q - - - - - - - 

Fitness: 39


In [ ]:
from operator import indexOf
import random

# Making random chromosomes
def random_chromosome(size):
    return [random.randint(0, size - 1) for _ in range(size)]


# Calculating fitness
def fitness(chromosome, maxFitness):
    horizontal_collisions = (
        sum([chromosome.count(queen) - 1 for queen in chromosome]) / 2
    )
    diagonal_collisions = 0

    n = len(chromosome)
    left_diagonal = [0] * (2 * n - 1)
    right_diagonal = [0] * (2 * n - 1)
    for i in range(n):
        left_diagonal[i + chromosome[i] - 1] += 1
        right_diagonal[len(chromosome) - i + chromosome[i] - 2] += 1

    diagonal_collisions = 0
    for i in range(2 * n - 1):
        counter = 0
        if left_diagonal[i] > 1:
            counter += left_diagonal[i] - 1
        if right_diagonal[i] > 1:
            counter += right_diagonal[i] - 1
        diagonal_collisions += counter

    # 28-(2+3)=23
    return int(maxFitness - (horizontal_collisions + diagonal_collisions))


# Doing cross_over between two chromosomes
def crossover(x, y):
    n = len(x)
    child = [0] * n
    for i in range(n):
        c = random.randint(0, 1)
        if c < 0.5:
            child[i] = x[i]
        else:
            child[i] = y[i]
    return child


# Randomly changing the value of a random index of a chromosome
def mutate(x):
    n = len(x)
    c = random.randint(0, n - 1)
    m = random.randint(0, n - 1)
    x[c] = m
    return x


# Calculating probability
def probability(chromosome, maxFitness):
    return fitness(chromosome, maxFitness) / maxFitness


# Roulette-wheel selection
def random_pick(population, probabilities):
    populationWithProbabilty = zip(population, probabilities)
    total = sum(w for c, w in populationWithProbabilty)
    r = random.uniform(0, total)
    upto = 0
    for c, w in zip(population, probabilities):
        if upto + w >= r:
            return c
        upto += w
    assert False, "Shouldn't get here"


# Genetic algorithm
def genetic_queen(population, maxFitness):
    mutation_probability = 0.1
    new_population = []
    sorted_population = []
    probabilities = []
    for n in population:
        f = fitness(n, maxFitness)
        probabilities.append(f / maxFitness)
        sorted_population.append([f, n])

    sorted_population.sort(reverse=True)

    # Elitism
    new_population.append(sorted_population[0][1])  # the best gen
    new_population.append(sorted_population[-1][1])  # the worst gen

    for i in range(len(population) - 2):

        chromosome_1 = random_pick(population, probabilities)
        chromosome_2 = random_pick(population, probabilities)

        # Creating two new chromosomes from 2 chromosomes
        child = crossover(chromosome_1, chromosome_2)

        # Mutation
        if random.random() < mutation_probability:
            child = mutate(child)

        new_population.append(child)
        if fitness(child, maxFitness) == maxFitness:
            break
    return new_population


# prints given chromosome
def print_chromosome(chrom, maxFitness):
    print(
        "Chromosome = {},  Fitness = {}".format(str(chrom), fitness(chrom, maxFitness))
    )


# prints given chromosome board
def print_board(chrom):
    board = []

    for x in range(nq):
        board.append(["x"] * nq)

    for i in range(nq):
        board[chrom[i]][i] = "Q"

    def print_board(board):
        for row in board:
            print(" ".join(row))

    print()
    print_board(board)


if __name__ == "__main__":
    POPULATION_SIZE = 500

    while True:
        # say N = 8
        nq = int(input("Please enter your desired number of queens (0 for exit): "))
        if nq == 0:
            break

        maxFitness = (nq * (nq - 1)) / 2  # 8*7/2 = 28
        population = [random_chromosome(nq) for _ in range(POPULATION_SIZE)]

        generation = 1
        while (
            not maxFitness in [fitness(chrom, maxFitness) for chrom in population]
            and generation < 200
        ):

            population = genetic_queen(population, maxFitness)
            if generation % 10 == 0:
                print("=== Generation {} ===".format(generation))
                print(
                    "Maximum Fitness = {}".format(
                        max([fitness(n, maxFitness) for n in population])
                    )
                )
            generation += 1

        fitnessOfChromosomes = [fitness(chrom, maxFitness) for chrom in population]

        bestChromosomes = population[
            indexOf(fitnessOfChromosomes, max(fitnessOfChromosomes))
        ]

        if maxFitness in fitnessOfChromosomes:
            print("\nSolved in Generation {}!".format(generation - 1))

            print_chromosome(bestChromosomes, maxFitness)

            print_board(bestChromosomes)

        else:
            print(
                "\nUnfortunately, we could't find the answer until generation {}. The best answer that the algorithm found was:".format(
                    generation - 1
                )
            )
            print_board(bestChromosomes)

Please enter your desired number of queens (0 for exit): 8

Solved in Generation 9!
Chromosome = [5, 2, 6, 3, 0, 7, 1, 4],  Fitness = 28

x x x x Q x x x
x x x x x x Q x
x Q x x x x x x
x x x Q x x x x
x x x x x x x Q
Q x x x x x x x
x x Q x x x x x
x x x x x Q x x
Please enter your desired number of queens (0 for exit): 8
=== Generation 10 ===
Maximum Fitness = 27

Solved in Generation 17!
Chromosome = [5, 2, 6, 1, 7, 4, 0, 3],  Fitness = 28

x x x x x x Q x
x x x Q x x x x
x Q x x x x x x
x x x x x x x Q
x x x x x Q x x
Q x x x x x x x
x x Q x x x x x
x x x x Q x x x
